In [4]:
import torch.nn as nn

class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
                nn.Conv2d(1, 64, kernel_size = 9, stride = 1, padding = 4),
                nn.ReLU())
        
        self.layer2 = nn.Sequential(
                nn.Conv2d(64, 32, kernel_size = 1, stride = 1, padding = 0),
                nn.ReLU())
        
        self.layer3 = nn.Sequential(
                nn.Conv2d(32, 1, kernel_size = 5, stride = 1, padding = 2))
        
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        return x

In [5]:
# Find total parameters and trainable parameters
model = SRCNN()
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

8,129 total parameters.
8,129 training parameters.


In [7]:
from torchsummary import summary

model = SRCNN()
summary(model, (1,32,32)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           5,248
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 32, 32, 32]           2,080
              ReLU-4           [-1, 32, 32, 32]               0
            Conv2d-5            [-1, 1, 32, 32]             801
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 0.03
Estimated Total Size (MB): 1.54
----------------------------------------------------------------
